In [7]:
from pathlib import Path
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.namespace import OWL, RDFS, XSD

# Start from the notebook's current directory
NB_DIR = Path.cwd().resolve()

# Walk up to find a directory that contains a 'src' folder
SRC_DIR = None
for p in [NB_DIR] + list(NB_DIR.parents):
    candidate = p / "assignment" / "src"
    if candidate.exists():
        SRC_DIR = candidate
        break
    candidate = p / "src"  # also allow projects that don't have the extra 'assignment' level
    if candidate.exists():
        SRC_DIR = candidate
        break

if SRC_DIR is None:
    raise FileNotFoundError(
        f"Could not find a 'src' directory above {NB_DIR}. "
        "If your project root is at e.g. .../assignment/, open the notebook there or set SRC_DIR manually."
    )

DATA_DIR = SRC_DIR / "data"

print("Notebook CWD:   ", NB_DIR)
print("Resolved SRC_DIR:", SRC_DIR)
print("Resolved DATA_DIR:", DATA_DIR)
print("TTL files found:", [p.name for p in SRC_DIR.glob("*.ttl")])


TTL = {
    "bfo":  SRC_DIR / "bfo-core.ttl",
    "ies":  SRC_DIR / "ies.ttl",
    "qudt": SRC_DIR / "qudt.ttl",
    "ccom": SRC_DIR / "ccom.ttl",
    "ccot": SRC_DIR / "ccot.ttl",
    "to":   SRC_DIR / "time.ttl",
}

OUT = {
    "bfo":  DATA_DIR / "bfo-axioms.xlsx",
    "ies":  DATA_DIR / "ies-axioms.xlsx",
    "qudt": DATA_DIR / "qudt-axioms.xlsx",
    "ccom": DATA_DIR / "ccom-axioms.xlsx",
    "ccot": DATA_DIR / "ccot-axioms.xlsx",
    "to":   DATA_DIR / "to-axioms.xlsx",
}

CLASS_AXIOM_QUERY = """

SELECT DISTINCT
  ?class
  ?direction         # "asSubject" or "asObject"
  ?predicate
  ?other             # object when class is subject; subject when class is object
  ?axiom             # a compact (predicate, other) representation
  ?otherLabel
WHERE {
  # Identify classes
  ?class a ?classType .
  VALUES ?classType { owl:Class rdfs:Class owl:ObjectProperty  }

  # Filter out deprecated classes (true as boolean or string)
  FILTER NOT EXISTS {
    ?class owl:deprecated ?d .
    FILTER( ?d = true || lcase(str(?d)) = "true" )
  }

  FILTER( !isBlank(?class) )

  # --- Case 1: triples where the class is the SUBJECT
  {
    BIND("asSubject" AS ?direction)
    ?class ?predicate ?other .
  }
  UNION
  # --- Case 2: triples where the class is the OBJECT
  {
    BIND("asObject" AS ?direction)
    ?other ?predicate ?class .
  }

  # A readable “pair” for the axiom (predicate, other)
  BIND( CONCAT(STR(?predicate), " :: ", STR(?other)) AS ?axiom )

  # Convenience label for the other node when it’s an IRI
  OPTIONAL {
    FILTER( isIRI(?other) )
    ?other rdfs:label ?otherLabel .
  }
}
ORDER BY ?class ?direction ?predicate ?other
"""

def run_local_select(ttl_path: Path, query: str) -> pd.DataFrame:
    g = Graph()
    g.parse(ttl_path)  # rdflib guesses Turtle from .ttl
    res = g.query(query)
    rows = []
    for row in res:
        d = row.asdict()  # {'iri': rdflib.term.URIRef(...), 'label': rdflib.term.Literal(...)} or missing
        class_term   = d.get("class")
        direction_term = d.get("direction")
        predicate_term = d.get("predicate")
        other_term = d.get("other")
        
# Build axiom string by joining non-None values
        axiom_parts = [str(term) for term in [direction_term, predicate_term, other_term] if term is not None]
        axiom = " ".join(axiom_parts) if axiom_parts else None

        rows.append({
            "iri":   str(class_term)   if class_term is not None else None,
            "axiom": axiom

        })
    return pd.DataFrame(rows, columns=["iri", "axiom"])

for key, ttl_path in TTL.items():
    if not ttl_path.exists():
        print(f"⚠️  Missing {ttl_path} — skipping {key}")
        continue
    print(f"Querying {ttl_path} …")
    df = run_local_select(ttl_path, CLASS_AXIOM_QUERY)
    df.to_excel(OUT[key], index=False)
    print(f"✅ Saved {len(df)} rows to {OUT[key]}")



Notebook CWD:    C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\notebooks
Resolved SRC_DIR: C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src
Resolved DATA_DIR: C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\data
TTL files found: ['bfo-core.ttl', 'ccom.ttl', 'ccot.ttl', 'ies.ttl', 'qudt.ttl', 'time.ttl']
Querying C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\bfo-core.ttl …
✅ Saved 847 rows to C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\data\bfo-axioms.xlsx
Querying C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\ies.ttl …
✅ Saved 4802 rows to C:\Users\gregd\Documents\UB\ontology tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\data\ies-axioms.xlsx
Qu